# Graph Learning
## Lab 6: Spectral Embedding

In this lab, you will learn to embed the nodes of a graph in a vector space of low dimension. We consider the  embedding based on the top eigenvectors of the transition matrix $P=D^{-1}A$.

## Import

In [ ]:
from IPython.display import SVG

In [ ]:
import numpy as np
from scipy import sparse
from matplotlib import pyplot as plt

In [ ]:
from sknetwork.data import load_netset, karate_club
from sknetwork.embedding import Spectral
from sknetwork.ranking import PageRank
from sknetwork.visualization import visualize_graph
from sklearn.cluster import KMeans
from sknetwork.visualization import visualize_dendrogram

In [ ]:
from scipy.cluster.hierarchy import linkage
import pandas as pd
import plotly.express as px
import nbformat

## Data

We will work on the following graphs (see the [NetSet](https://netset.telecom-paris.fr/) collection for details):
* Openflights (graph)
* WikiVitals (directed graph and bipartite graph)

In [ ]:
openflights = load_netset('openflights')
wikivitals = load_netset('wikivitals')

## 1. Graphs

## Karate Club


We first consider the spectral embedding of the [karate club graph](https://en.wikipedia.org/wiki/Zachary%27s_karate_club).

In [ ]:
dataset = karate_club(metadata=True)

In [ ]:
adjacency = dataset.adjacency
position = dataset.position
labels_true = dataset.labels

In [ ]:
image = visualize_graph(adjacency, position, labels=labels_true)
SVG(image)

## To do

* Display the spectrum of the transition matrix (e.g., first 20 eigenvalues).
* What does the spectrum suggest?
* Display the graph with some eigenvectors.
* Display the embedding of the graph in dimension 2.
* Compare the clusters obtained with the sign of the first component of the embedding to the ground-truth clusters.

In [ ]:
spectral = Spectral(20, normalized=False)

In [ ]:
spectral.fit(adjacency)

In [ ]:
# eigenvalues (adding the first)
eigenvalues = [1] + list(spectral.eigenvalues_)

In [ ]:
# 1. Display the spectrum of the transition matrix (first 20 eigenvalues)
plt.figure(figsize=(10, 6))
plt.scatter(np.arange(len(eigenvalues)) + 1, eigenvalues, color='r', s=100, edgecolor='k', lw=1)
plt.ylim(-1.1, 1.1)
plt.axhline(0, color='k', linestyle='-', linewidth=0.5)
plt.title("Eigenvalue Spectrum of the Graph Laplacian", fontsize=14, pad=20)
plt.xlabel("Eigenvalue Index (k)", fontsize=12)
plt.ylabel("Eigenvalue (λₖ)", fontsize=12)

The spectrum suggest that there are 4 clusters, as there are 4 dominant eigenvalues, this is was we also got when we have done the Louvain algorithm.

In [ ]:
# eigenvectors
eigenvectors = spectral.eigenvectors_

In [ ]:
# Displaying eigenvector 0
image = visualize_graph(adjacency, position, scores=eigenvectors[:, 0])
SVG(image)

In [ ]:
# Displaying eigenvector 1
image = visualize_graph(adjacency, position, scores=eigenvectors[:, 1])
SVG(image)

In [ ]:
plt.figure(figsize=(7, 7), facecolor='white', dpi=100)

for label in np.unique(labels_true):
    plt.scatter( eigenvectors[labels_true == label, 0], eigenvectors[labels_true == label, 1], s=150, c=['r', 'b'][label], edgecolor='k', linewidth=0.8)
plt.axvline(x=0, c='k', linestyle='--', linewidth=0.8, alpha=0.7)
plt.axhline(y=0, c='k', linestyle='--', linewidth=0.8, alpha=0.7)
plt.title("Spectral Embedding (2D)", fontsize=14, pad=15)
plt.xlabel("First Eigenvector", fontsize=12)
plt.ylabel("Second Eigenvector", fontsize=12)
plt.grid(True, linestyle=':', alpha=0.4)
plt.tight_layout()
legend_labels = ['Community 0', 'Community 1']
plt.legend(legend_labels, frameon=True, facecolor='white', edgecolor='none')
plt.show()

In [ ]:
labels_pred = (eigenvectors[:,0] > 0).astype(int)
image = visualize_graph(adjacency, position, labels=labels_pred)
SVG(image)

In [ ]:
print(" Accuracy :" ,np.mean(labels_pred==labels_true))

## Openflights


We now consider a larger graph. We use spectral embedding in dimension 20 to cluster the graph by k-means in the embedding space.

In [ ]:
dataset = openflights

In [ ]:
adjacency = dataset.adjacency
position = dataset.position
names = dataset.names

In [ ]:
image = visualize_graph(adjacency, position, width=800, height=400, node_size=3, display_edges=False)
SVG(image)

## To do

* Display the same world map with 8 clusters found by k-means in the embedding space. You can use ``scikit-learn``for k-means.
* Do the same without normalization on the unit sphere (``normalized=False``).<br> Interpret the results. <br>**Hint:** Compute the Euclidean norm of the centroid of each cluster in the embedding space.

In [ ]:
spectral = Spectral(20, normalized=True)

In [ ]:
embedding_norm = spectral.fit_transform(adjacency)

In [ ]:
# Displaying world map with 8 clusters
kmeans = KMeans(8, n_init=20)
labels_norm = kmeans.fit_predict(embedding_norm)
image = visualize_graph(adjacency, position, width=800, height=400, node_size=3, labels=labels_norm, display_edges=False)
SVG(image)

In [ ]:
spectral = Spectral(20, normalized=False)
embedding = spectral.fit_transform(adjacency)
kmeans = KMeans(8, n_init=20)
labels = kmeans.fit_predict(embedding)
image = visualize_graph(adjacency, position, width=800, height=400, node_size=3, labels=labels, display_edges=False)
SVG(image)

In [ ]:
def analyze_clusters(embedding, labels, normalized):
    print(f"\nAnalysis for {'normalized' if normalized else 'non-normalized'} embedding:")
    for i in range(8):
        cluster_points = embedding[labels == i]
        centroid = kmeans.cluster_centers_[i]
        avg_norm = np.mean(np.linalg.norm(cluster_points, axis=1))
        centroid_norm = np.linalg.norm(centroid)

        print(f"Cluster {i}:")
        print(f"  • Avg point norm: {avg_norm:.3f}")
        print(f"  • Centroid norm: {centroid_norm:.3f}")
        print(f"  • Size: {len(cluster_points)} points")

analyze_clusters(embedding_norm, labels_norm, normalized=True)
analyze_clusters(embedding, labels, normalized=False)

Normalized embedding clusters represent directional patterns on a unit sphere, where all points have equal magnitude but vary in angular position. Non-normalized embedding clusters reflect both magnitude and direction, revealing groups with similar absolute distances from the origin—higher centroid norms indicate more distinct clusters, while lower norms suggest overlapping or less distinct groups.

## 2. Directed graphs and bipartite graphs

We now work on directed graph and bipartite graphs. We measure proximity between nodes in the embedding space in terms of [cosine similarity](https://en.wikipedia.org/wiki/Cosine_similarity). Equivalently, we project the vectors on the unit sphere.

In [ ]:
spectral = Spectral(20, normalized=True)

## Wikipedia Vitals

In [ ]:
dataset = wikivitals

In [ ]:
adjacency = dataset.adjacency
biadjacency = dataset.biadjacency
names = dataset.names
words = dataset.names_col
labels = dataset.labels
names_labels = dataset.names_labels
labels_hierarchy = dataset.labels_hierarchy
names_labels_hierarchy = dataset.names_labels_hierarchy

## To do

We first consider the spectral embedding of the graph of links in dimension 20.

* List the 20 articles that are closest to **Vincent van Gogh** in terms of cosine similarity in the embedding space. Compare with the top articles obtained with Personalized PageRank.
* Display the 3D-plot of each of the 11 labels in the embedding space (top 3 dimensions). <br>You might represent each label by a point located at the centroid of the corresponding articles, with a size proportional to the number of articles. Use ``plotly`` for an interactive plot. Interpret the results.
* Display the dendrogram of the hierarchical clustering of the top-100 articles on **Arts** (in terms of Personalized PageRank). You might use the [Ward method](https://en.wikipedia.org/wiki/Ward%27s_method) in the embedding space. Comment the results.

In [ ]:
spectral = Spectral(20)
embedding = spectral.fit_transform(adjacency)
node = np.flatnonzero(names=='Vincent van Gogh')[0]
# cosine similarity
scores = embedding.dot(embedding[node])
print("Top 20 articles by cosine similarity :", names[np.argsort(-scores)[:20]])
van_gogh_idx = np.where(names == "Vincent van Gogh")[0][0]
pagerank = PageRank()
scores_ppr = pagerank.fit_predict(adjacency, weights={van_gogh_idx: 1})
top_pagerank = np.argsort(- scores_ppr)[:20]
print("Top 20 articles by PPR : ", names[top_pagerank])



In [ ]:
labels_unique, counts = np.unique(labels, return_counts=True)
embedding_label = np.array([np.mean(embedding[labels==label], axis=0) for label in labels_unique])
dataframe = pd.DataFrame(embedding_label[:, :3], columns=list('xyz'))
dataframe['category'] = names_labels[labels_unique]
dataframe['count'] = counts
fig = px.scatter_3d(dataframe, x='x', y='y', z='z', text='category', color='category', size='count', size_max=100, opacity=0.5, title='3D Spectral Embedding of Clusters' )
fig.update_layout(showlegend=False)

In [ ]:
pagerank = PageRank()
scores = pagerank.fit_predict(adjacency, labels==0)
scores *= labels==0
top = np.argsort(-scores)[:100]
dendrogram = linkage(embedding[top], method='ward')
SVG(visualize_dendrogram(dendrogram, names=names[top], rotate=True, rotate_names=True, height=1000))

## To do

* Repeat the same experiments on the bipartite graph between articles and words.
* List the 10 articles and the 10 words that are closest to the word **painting** in the embedding space.

In [ ]:
embedding = spectral.fit_transform(biadjacency)
embedding_words = spectral.embedding_col_

# cosine similarity with articles
scores = embedding.dot(embedding_words[words == "painting"].ravel())
print("\n=== Top 10 Articles Related to 'painting' ===")
for i, idx in enumerate(np.argsort(-scores)[:10], 1):
    print(f"{i}. {names[idx]}")

# cosine similarity with words
scores = embedding_words.dot(embedding_words[words == "painting"].ravel())
print("\n=== Top 10 Words Related to 'painting' ===")
for i, idx in enumerate(np.argsort(-scores)[:10], 1):
    print(f"{i}. {words[idx]}")

## To do

* Prove that the average cosine similarity between vectors in some set $S$ is equal to the square norm of the centroid of $S$.

$$
S=\{v_1,\ldots,v_n\}
$$
$$
\mu = \frac 1 n \sum_{i=1}^n v_i
$$

For the following questions, first consider the graph of links, then the bipartite graph between articles and words:

* Compute the average cosine similarity between articles of the **Mammals** category (see hierarchical labels).
* Compare with the expected cosine similarity between two articles sampled uniformly at random.
* Defining a category as **topical** if its average cosine similarity is close to 1, rank the 11 categories (Arts, History,...) by topicality.
* List the 10 most topical and the 10 less topical hierarchical categories having at least 10 articles (like **Mammals**). Comment the results.

**Proof**:
$$
\frac{1}{n^2}\sum_{i,j} v_i^T v_j
= \frac{1}{n^2}\left(\sum_i v_i\right)^T\left(\sum_j v_j\right)
= \left\|\frac{1}{n}\sum_i v_i\right\|^2
= \|\mu\|^2
$$

In [ ]:
biadjacency = dataset.biadjacency
spectral = Spectral(20)
embedding = spectral.fit_transform(biadjacency)

mammal_category_id = next(
    label for label, name in enumerate(names_labels_hierarchy)
    if name.endswith("Mammals")
)

def get_average_cosine(mask):
    return np.linalg.norm(np.mean(embedding[mask], axis=0))**2

mammals_avg_cosine = get_average_cosine(labels_hierarchy == mammal_category_id)

print(f"Average cosine similarity for Mammals: {mammals_avg_cosine:.4f}")


In [ ]:
expected_random = get_average_cosine(labels_hierarchy >= 0)
print(f"Expected random: {expected_random:.4f}")

In [ ]:
scores = np.array([get_average_cosine(labels==label) for label in np.unique(labels)])
print("Ranked Categories by Topicality:")
print("--------------------------------")
for i, label in enumerate(np.argsort(-scores), 1):
    print(f"{i:2d}. {scores[label]:.3f} - {names_labels[label]}")

In [ ]:
unique_labels, counts = np.unique(labels_hierarchy, return_counts=True)
valid_labels = unique_labels[counts >= 10]

hier_scores = np.array([get_average_cosine(labels_hierarchy==label) for label in valid_labels])
hier_names = [names_labels_hierarchy[label] for label in valid_labels]

print("Top 10 Most Topical:")
for i in np.argsort(-hier_scores)[:10]:
    print(f"{hier_scores[i]:.3f} - {hier_names[i]}")

print("\nTop 10 Least Topical:")
for i in np.argsort(hier_scores)[:10]:
    print(f"{hier_scores[i]:.3f} - {hier_names[i]}")